# Primeiro Trabalho Computacional. Regressão Linear na qualidade de cerveja. 



## 1. Introdução

Está fornecido um arquivo csv (valores separados por vírgula) contendo características químicas de amostra de cerveja, e sua qualidade avaliada por especialistas, em uma escala de 6 a 10. Seu objetivo é avaliar um sistema de regressão linear sobre este problema. 

Os módulos, classes e definições contidas aqui devem ser vistas como sugestão. Desde que você use Python, apresente seu trabalho como um notebook e responda às perguntas, os requisitos estão satisfeitos. Eu também não tenho objeções ao uso de código encontrado na internet ou via chatGPT, mas use-os por sua conta e risco. 

## 2. Classes e Métodos

Com estes módulos, classes e métodos que se seguem, o laço principal de treinamento fica muito simples. Lembre-se de ativar o d2l antes de invocar o notebook. Estou usando também o módulo sklearn para a divisão do arquivo em treinamento e validação. É preciso instalá-lo com `pip install sklearn` no terminal. 

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from d2l import tensorflow as d2l
import random



2024-04-27 19:52:17.356970: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-27 19:52:17.385228: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-27 19:52:17.385806: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 19:52:17.944059: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class LinearRegression(d2l.Module):  #@save
    """The linear regression model implemented with high-level APIs."""
    """Do d2l"""
    def __init__(self, lr):
        super().__init__()
        self.save_hyperparameters()
        initializer = tf.initializers.RandomNormal(stddev=0.01)
        self.net = tf.keras.layers.Dense(1, kernel_initializer=initializer)

In [3]:
class LinearRegressionData(d2l.DataModule):  #@save
    """Data for linear regression."""
    def __init__(self, nInputs,data,num_train=5197, num_val=1300,
                 batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        self.X=data[:,0:nInputs]
        self.y=data[:,nInputs]

In [4]:
@d2l.add_to_class(d2l.DataModule)  #@save
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    shuffle_buffer = tensors[0].shape[0] if train else 1
    return tf.data.Dataset.from_tensor_slices(tensors).shuffle(
        buffer_size=shuffle_buffer).batch(self.batch_size)

@d2l.add_to_class(LinearRegressionData)  #@save
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)


## 3.  Leitura do Arquivo e Pré-processamento

Existem 7 variáveis de entrada e 1 alvo (Quality_Score). Há 25157 exemplos, e eles não são equilibrados (não existe a mesmo quantidade para cada nível de qualidade). 
| Fermentation_Time | Temperature      | pH_Level         | Gravity          | Alcohol_Content  | Bitterness | Color | Quality_Score    |
|-------------------|------------------|------------------|------------------|------------------|------------|-------|---|
| 16                | 24.2042508570699 | 5.28984544760956 | 1.0395041267302  | 5.37084215955344 | 20         | 5     | 8 |
| 13                | 18.0867629472595 | 5.27564338275619 | 1.05981895169872 | 5.09605308279763 | 36         | 14    | 7 |
| 12                | 15.5393326691165 | 4.77801562324598 | 1.03747570954872 | 4.82473712095918 | 30         | 10    | 8 |
| 17                | 16.4184891039432 | 5.34526058554619 | 1.05243142516949 | 5.509243080798   | 48         | 18    | 9 |
| 18                | 19.1449076543385 | 4.86185374113861 | 1.05429611494823 | 5.13362468426324 | 57         | 13    | 7 |
| 10                | 17.4246143933758 | 5.29178662190806 | 1.0646620426822  | 4.85917102161423 | 45         | 9     | 8 |
| ...               | ...              | ...              | ...              | ...              | ...        | ...   | ... |

Você deve ainda usar a função `train_test_split` (o outra) para separar estes arquivos em treinamento e validação. Sugiro 20% para a validação. 

In [5]:
train_data, val_data = train_test_split(data, test_size=0.2)
train_data.shape, val_data.shape


NameError: name 'data' is not defined

Finalmente, lembre-se que a função `pd.read_csv` cria um dataframe, não uma matriz. Verifique como transformar em uma matriz, que deve conter todos os exemplos de treinamento e em seguida todos os exemplos de validação. Sua dimensão final será 25157 por 8 (incluindo a variável alvo).


## 4. Métodos para treinamento

Estes métodos definem como se dá o passo à frente (processamento da rede), qual a função de custo e otimizador.

In [ ]:
@d2l.add_to_class(LinearRegression)  #@save
def forward(self, X):
    return self.net(X)

In [ ]:
@d2l.add_to_class(LinearRegression)  #@save
def loss(self, y_hat, y):
    fn = tf.keras.losses.MeanSquaredError()
    return fn(y, y_hat)

In [ ]:
@d2l.add_to_class(LinearRegression)  #@save
def configure_optimizers(self):
    return tf.keras.optimizers.SGD(self.lr)

### 4.1. Treinamento

Com esses passos, o laço de treinamento é bem simples. Veja o exemplo visto em sala. 
ATENÇÃO: Os dados têm variância (escala) desigual. Como nada limita a saída do modelo de regressão, uma taxa de aprendizado alta pode levar à divergência. Se o gráfico do erro fica em branco ou o treinamento leva tempo demais, este é provavelmente o caso. Experimente diferentes valores de taxa, em diferentes ordens de grandeza, para solucionar isso (10^-4, 10^-3, 10^-2, 10^-1...).

## 5. Avaliações e análise

Terminado o treinamento, houve (espera-se) uma minimização do erro quadrático médio. Mas outras medidas de desempenho podem ser usadas. 

Para avaliar o modelo, pode ser útil a seguinte função: 

``prediction=model.call(inputs)``

Supondo que você instanciou seu modelo como "model". Para fazer operações (cálculo de médias e erros), pode ser necessário mudar (reshape) algum tensor. Lembre-se de usar os dados de VALIDAÇÃO. Uma média de vários treinamentos com condiçoes iniciais (e partições da base dados) diferentes nos dá um resultado mais confiável. 

### 5.1. Erro absoluto médio e Preditor Trivial

Qual o erro absoluto médio do seu preditor? Como a categoria é dada por inteiros de 6 a 10, esperamos, no mínimo, um erro menor do que 1, ou o sistema frequentemente erraria por mais de uma categoria. 

Por vezes queremos saber se, depois de todo este trabalho temos algo superior a um preditor trivial ou ingênuo. O preditor trivial atribui como saída a média da função que se quer prever. 

Compare os erros absolutos médios. O seu regressor é melhor do que uma simples média da saída?

__Atenção__: Este problema é não-linear, e fizemos pouco pré-processamento com os dados. Não se espera um bom desempenho com este primeiro modelo simples. Ele deve, no entanto, ser melhor que o preditor trivial. 

### 5.2. Classificação por arredondamento

Como a classificação é um índice inteiro, podemos pensar neste caso como um problema de classificação em 5 classes, e não de regressão. Veremos ferramentas mais indicadas para esta classe de problemas, mas uma solução trivial seria arredondar a saída do preditor e verificar quantas classes acertamos. 

Compare a taxa de acerto (vamos chamar isso acurácia mais à frente) contra o preditor-classificador trivial. Um preditor aleatório (que atribui um número de 6 a 10 aleatoriamente) deveria ter uma taxa de acerto de 20%.

__Atenção__: de novo, você deve estar acima do preditor trivial, e acima dos 20%. 

### 5.3. Correlação

Outra medida interessante para saber o quanto nosso preditor absorveu a informação contida no arquivo de dados é a __correlação__ entre saída desejada e obtida. Este é um número de -1 a 1, -1 indicando uma relação determinística negativa (quando a saída do preditor é baixa, o especialista deu nota alta e vice_versa), 0 é ausência de correlação (a saída de um não nos permite dizer nada sobre o outro) e +1 significa que o preditor substitui o especialista sem qualquer erro. Gostaríamos, é claro, de um valor próximo a 1. Observe que o preditor aleatório e também o trivial têm correlação zero com a saída desejada. 

### 5.4. Melhoramentos (extra)

Não se espera mesmo um bom desempenho neste caso, mas algumas coisas podem ajudar. Não são requisitos para o trabalho, mas serão considerados pontos extra. 

* Variações no número de épocas, tamanho do "minibatch", taxas de aprendizado.
* Normalização das entradas (para que tenham média zero e desvio-padrão 1).
* (mais difícil) Uma análise estatística que elimine variáveis pouco informativas, ou proponha uma combinação polinomial mais informativa de algumas variáveis.

  

